### Getting data for a single stock and the steps to achieve the end outcome

In [ ]:
import yfinance as yf
import pandas as pd

# Step 1: Download 5 years of data for TCS
stock = "TCS.NS"
data = yf.download(stock, period="5y", interval="1d")
data["Prev_Close"] = data["Close"].shift(1)
data = data.dropna().copy()


<ipython-input-1-3901739518>:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stock, period="5y", interval="1d")
[*********************100%***********************]  1 of 1 completed


prompt: what does .shift(1) do in the above code?

.shift(1) on a pandas Series or DataFrame shifts the data by 1 period.
In this specific case, data["Close"].shift(1) shifts the "Close" column down by one row.
This means that for each row, the value in the "Prev_Close" column will be the "Close" price from the previous day.
This is commonly used to calculate daily returns or compare the current day's price to the previous day's price.

In [ ]:
data.head()

Price,Close,High,Low,Open,Volume,Prev_Close
Ticker,TCS.NS,TCS.NS,TCS.NS,TCS.NS,TCS.NS,
Date,,,,,,
2020-06-19,1845.080322,1886.050044,1833.709899,1842.734062,5681758,1839.214722
2020-06-22,1830.054932,1851.983690,1822.113735,1849.953253,3196138,1845.080322
2020-06-23,1836.688232,1844.539211,1813.857052,1829.198132,4205944,1830.054932
2020-06-24,1842.914795,1862.813121,1833.349203,1835.515024,2560937,1836.688232
2020-06-25,1819.361450,1858.977547,1813.179921,1836.417140,3043931,1842.914795


In [ ]:
data.shape

(1239, 6)

In [ ]:
data.iloc[2 - 1]["Close"]

,2020-06-22
Ticker,
TCS.NS,1830.054932


In [ ]:
# Step 2: List to store trades
trades = []



In [ ]:
data.iloc[2]["Open"][0]

<ipython-input-24-1799209704>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.iloc[2]["Open"][0]


np.float64(1829.1981318738394)

In [ ]:
data.iloc[2]["High"][0]

<ipython-input-28-2016941829>:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  data.iloc[2]["High"][0]


np.float64(1844.5392114208819)

In [ ]:
data.index[2]

Timestamp('2020-06-23 00:00:00')

In [ ]:
# Step 3: Loop through data to detect gap-down recovery pattern
for i in range(2, len(data)):
    prev_close = data.iloc[i - 1]["Close"][0]
    today_open = data.iloc[i]["Open"][0]

    if today_open <= prev_close * 0.98:
        for j in range(i, min(i + 60, len(data))):
            high = data.iloc[j]["High"][0]
            if high >= prev_close:
                if high >= prev_close * 1.02:
                    entry_price = round(prev_close * 1.02, 2)
                    entry_date = data.index[j]

                    for k in range(j, min(j + 60, len(data))):
                        high_k = data.iloc[k]["High"][0]
                        low_k = data.iloc[k]["Low"][0]
                        close_k = data.iloc[k]["Close"][0]
                        exit_date = data.index[k]

                        if high_k >= entry_price * 1.10:
                            exit_price = round(entry_price * 1.10, 2)
                            outcome = "Target Hit"
                            break
                        elif low_k <= entry_price * 0.95:
                            exit_price = round(entry_price * 0.95, 2)
                            outcome = "Stop Loss"
                            break
                        elif k == j + 59:
                            exit_price = round(close_k, 2)
                            outcome = "Max Hold"
                            break

                    trades.append({
                        "Stock": stock,
                        "Entry Date": entry_date.date(),
                        "Entry Price": entry_price,
                        "Exit Date": exit_date.date(),
                        "Exit Price": exit_price,
                        "P&L (%)": round((exit_price - entry_price) / entry_price * 100, 2),
                        "Days Held": (exit_date - entry_date).days,
                        "Outcome": outcome
                    })
                    break



<ipython-input-31-173625938>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_close = data.iloc[i - 1]["Close"][0]
<ipython-input-31-173625938>:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_open = data.iloc[i]["Open"][0]
<ipython-input-31-173625938>:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  high = data.iloc[j]["High"][0]
<ipython-input-31-173625938>:15: FutureWarning: Series.__getitem__ treating keys as position

Whats actually happening? :

1.  **`for i in range(2, len(data)):`**: This is the main loop that iterates through the `data` DataFrame.
    *   `range(2, len(data))` creates a sequence of numbers starting from 2 up to (but not including) the total number of rows in the `data` DataFrame.
    *   We start from index 2 because the pattern we're looking for requires comparing the current day's data with the previous day's close, and the previous day's close requires the day before that (hence, we need at least two prior data points).

2.  **`prev_close = data.iloc[i - 1]["Close"][0]`**:
    *   `data.iloc[i - 1]` accesses the row *before* the current row (at index `i-1`) using integer-location based indexing (`iloc`).
    *   `["Close"]` selects the "Close" column from that row.
    *   `[0]` accesses the first element of the selected "Close" value. Although "Close" should be a single value per row, accessing it this way ensures you get the scalar value. This retrieves the closing price of the previous day.

3.  **`today_open = data.iloc[i]["Open"][0]`**:
    *   `data.iloc[i]` accesses the current row (at index `i`).
    *   `["Open"]` selects the "Open" column from the current row.
    *   `[0]` accesses the scalar opening price for the current day.

4.  **`if today_open <= prev_close * 0.98:`**: This is the first condition for detecting the "gap-down" part of the pattern.
    *   It checks if the current day's opening price (`today_open`) is less than or equal to 98% (0.98) of the previous day's closing price (`prev_close`).
    *   If this condition is **True**, it indicates a significant gap down from the previous day's close. The code then proceeds to look for a recovery.

5.  **`for j in range(i, min(i + 60, len(data))):`**: This is the *nested* loop that searches for the "recovery" part of the pattern *after* a gap down has been detected on day `i`.
    *   `range(i, min(i + 60, len(data)))` iterates through the data starting from the current day `i` up to a maximum of 60 days *or* until the end of the DataFrame, whichever comes first. This limits the search for the recovery to a reasonable timeframe after the gap down.

6.  **`high = data.iloc[j]["High"][0]`**:
    *   Inside the recovery search loop, this gets the high price for the day currently being checked (`j`).

7.  **`if high >= prev_close:`**: This checks if the high price on day `j` has reached or exceeded the `prev_close` (the closing price of the day *before* the gap down). This is the first part of the recovery confirmation.

8.  **`if high >= prev_close * 1.02:`**: This is a more stringent condition. If the high on day `j` reaches or exceeds 102% (1.02) of the `prev_close`, it's considered a strong enough recovery to trigger a potential trade entry.

9.  **`entry_price = round(prev_close * 1.02, 2)`**: If the strong recovery condition is met, the `entry_price` is set to 102% of the `prev_close`, rounded to two decimal places.

10. **`entry_date = data.index[j]`**: The `entry_date` is set to the date corresponding to the current day `j` where the recovery condition was met.

11. **`for k in range(j, min(j + 60, len(data))):`**: This is another *nested* loop, initiated *after* a potential entry is identified on day `j`. This loop searches for an exit condition (target hit, stop loss, or maximum hold) starting from the `entry_date` (`j`) for up to 60 days.

12. **`high_k = data.iloc[k]["High"][0]`**, **`low_k = data.iloc[k]["Low"][0]`**, **`close_k = data.iloc[k]["Close"][0]`**: Inside the exit search loop, these lines get the high, low, and close prices for the day currently being checked (`k`).

13. **`exit_date = data.index[k]`**: The `exit_date` is updated in each iteration of this loop. It will hold the date of the day where one of the exit conditions is met.

14. **`if high_k >= entry_price * 1.10:`**: This checks for the "Target Hit" condition. If the high price on day `k` is greater than or equal to 110% (1.10) of the `entry_price`, the target is hit.
    *   **`exit_price = round(entry_price * 1.10, 2)`**: Sets the `exit_price` to the target price.
    *   **`outcome = "Target Hit"`**: Sets the `outcome` string.
    *   **`break`**: Exits the *innermost* loop (the `k` loop) because an exit condition has been met.

15. **`elif low_k <= entry_price * 0.95:`**: This checks for the "Stop Loss" condition. If the high price target wasn't hit, it checks if the low price on day `k` is less than or equal to 95% (0.95) of the `entry_price`.
    *   **`exit_price = round(entry_price * 0.95, 2)`**: Sets the `exit_price` to the stop loss price.
    *   **`outcome = "Stop Loss"`**: Sets the `outcome` string.
    *   **`break`**: Exits the `k` loop.

16. **`elif k == j + 59:`**: This checks for the "Max Hold" condition. If neither the target nor the stop loss was hit within the 60-day exit window (meaning the loop reached its last iteration `k == j + 60 - 1`, which is `k == j + 59`), the trade is closed at the closing price of that day.
    *   **`exit_price = round(close_k, 2)`**: Sets the `exit_price` to the closing price on the last day of the hold period.
    *   **`outcome = "Max Hold"`**: Sets the `outcome` string.
    *   **`break`**: Exits the `k` loop.

In [ ]:
# Step 4: Convert to DataFrame and show result
df = pd.DataFrame(trades)
print(df.head())

    Stock  Entry Date  Entry Price   Exit Date  Exit Price  P&L (%)  \
0  TCS.NS  2022-03-09      3343.70  2022-04-19     3176.52    -5.00   
1  TCS.NS  2022-03-08      3306.69  2022-05-09     3141.36    -5.00   
2  TCS.NS  2022-08-11      3173.63  2022-08-25     3014.95    -5.00   
3  TCS.NS  2022-11-11      3051.60  2023-02-03     3307.30     8.38   
4  TCS.NS  2022-11-11      3058.37  2023-02-03     3307.30     8.14   

   Days Held    Outcome  
0         41  Stop Loss  
1         62  Stop Loss  
2         14  Stop Loss  
3         84   Max Hold  
4         84   Max Hold  


In [ ]:
df.shape

(7, 8)

### Nifty 500 companies with a .csv file

In [5]:
data2 = pd.read_csv("ind_nifty500list.csv")
data2.head()

,Company Name,Industry,Symbol,Series,ISIN Code
0,360 ONE WAM Ltd.,Financial Services,360ONE,EQ,INE466L01038
1,3M India Ltd.,Diversified,3MINDIA,EQ,INE470A01017
2,ABB India Ltd.,Capital Goods,ABB,EQ,INE117A01022
3,ACC Ltd.,Construction Materials,ACC,EQ,INE012A01025
4,ACME Solar Holdings Ltd.,Power,ACMESOLAR,EQ,INE622W01025


In [6]:
data2.shape

(503, 5)

In [18]:
data2[data2["Symbol"].apply(lambda x : x.startswith("TCS"))]

,Company Name,Industry,Symbol,Series,ISIN Code
444,Tata Consultancy Services Ltd.,Information Technology,TCS,EQ,INE467B01029


In [22]:
# we will go with the first 300 nifty
nifty_300_symbol = data2["Symbol"].unique()[0:300]

### Looping through all 300 nifty to get pricing pattern and P&L % with number of days

In [23]:
# List to store trades
nifty_300_trades = []

In [25]:
# looping through 300 nifty and collecting the information in nifty_300_trades

for sym in nifty_300_symbol:
  sym = sym + ".NS"

  # Step 1: Download 5 years of data for TCS
  stock = sym
  data = yf.download(stock, period="5y", interval="1d")
  data["Prev_Close"] = data["Close"].shift(1)
  data = data.dropna().copy()

  for i in range(2, len(data)):
    prev_close = data.iloc[i - 1]["Close"][0]
    today_open = data.iloc[i]["Open"][0]

    if today_open <= prev_close * 0.98:
        for j in range(i, min(i + 60, len(data))):
            high = data.iloc[j]["High"][0]
            if high >= prev_close:
                if high >= prev_close * 1.02:
                    entry_price = round(prev_close * 1.02, 2)
                    entry_date = data.index[j]

                    for k in range(j, min(j + 60, len(data))):
                        high_k = data.iloc[k]["High"][0]
                        low_k = data.iloc[k]["Low"][0]
                        close_k = data.iloc[k]["Close"][0]
                        exit_date = data.index[k]

                        if high_k >= entry_price * 1.10:
                            exit_price = round(entry_price * 1.10, 2)
                            outcome = "Target Hit"
                            break
                        elif low_k <= entry_price * 0.95:
                            exit_price = round(entry_price * 0.95, 2)
                            outcome = "Stop Loss"
                            break
                        elif k == j + 59:
                            exit_price = round(close_k, 2)
                            outcome = "Max Hold"
                            break

                    nifty_300_trades.append({
                        "Stock": stock,
                        "Entry Date": entry_date.date(),
                        "Entry Price": entry_price,
                        "Exit Date": exit_date.date(),
                        "Exit Price": exit_price,
                        "P&L (%)": round((exit_price - entry_price) / entry_price * 100, 2),
                        "Days Held": (exit_date - entry_date).days,
                        "Outcome": outcome
                    })
                    break





/tmp/ipython-input-25-166261852.py:8: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(stock, period="5y", interval="1d")
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-25-166261852.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prev_close = data.iloc[i - 1]["Close"][0]
/tmp/ipython-input-25-166261852.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  today_open = data.iloc[i]["Open"][0]
/tmp/ipython-input-25-166261852.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version

In [27]:
#Convert to DataFrame and show result
df = pd.DataFrame(nifty_300_trades)
print(df.head())

       Stock  Entry Date  Entry Price   Exit Date  Exit Price  P&L (%)  \
0  360ONE.NS  2020-07-30       209.53  2020-08-11      230.48     10.0   
1  360ONE.NS  2020-08-11       229.01  2020-08-12      251.91     10.0   
2  360ONE.NS  2020-08-17       230.81  2020-08-31      219.27     -5.0   
3  360ONE.NS  2021-08-06       359.27  2021-08-06      341.31     -5.0   
4  360ONE.NS  2021-08-31       366.48  2021-09-21      348.16     -5.0   

   Days Held     Outcome  
0         12  Target Hit  
1          1  Target Hit  
2         14   Stop Loss  
3          0   Stop Loss  
4         21   Stop Loss  


In [31]:
df.to_excel("nifty_300_trades2.xlsx", index=False)